### Putting it all together

In [2]:
import torch
import numpy as np
torch.set_printoptions(edgeitems=2, precision=6, linewidth=75, sci_mode=False)

在前面的几节中，我们一直在尽力手工完成大部分工作。我们已经探讨了标记器的工作原理，并研究了标记化、到输入id的转换、padding、truncation和truncation。

然而，正如我们在第2节中看到的，Transformers API可以用一个高级函数为我们处理所有这些，我们将在这里深入研究这个函数。当你直接在句子上调用你的标记器时，你会得到准备通过你的模型的输入:

In [3]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)
model_inputs

{'input_ids': [101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

这里，model_inputs变量包含模型正常运行所需的所有内容。对于DistilBERT，这包括input ID和attention mask。其他接受额外输入的模型也将由tokenizer对象输出这些输入。

正如我们将在下面的一些示例中看到的，这个方法非常强大。首先，它可以标记单个序列:

In [ ]:
sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)

它还可以一次处理多个序列，而API没有变化:

In [ ]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

model_inputs = tokenizer(sequences)

它可以根据几个目标进行填充:

In [ ]:
# Will pad the sequences up to the maximum sequence length
model_inputs = tokenizer(sequences, padding="longest")

# Will pad the sequences up to the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, padding="max_length")

# Will pad the sequences up to the specified max length
model_inputs = tokenizer(sequences, padding="max_length", max_length=8)

它也可以截断序列:

In [ ]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# Will truncate the sequences that are longer than the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, truncation=True)

# Will truncate the sequences that are longer than the specified max length
model_inputs = tokenizer(sequences, max_length=8, truncation=True)

标记器对象可以处理到特定框架tensor的转换，然后将其直接发送到模型。例如，在下面的代码示例中，我们提示标记器返回来自不同框架的张量——“pt”返回PyTorch张量，“tf”返回TensorFlow张量，“np”返回NumPy数组:

In [ ]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# Returns PyTorch tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="pt")

# Returns TensorFlow tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="tf")

# Returns NumPy arrays
model_inputs = tokenizer(sequences, padding=True, return_tensors="np")

#### Special tokens

如果我们看一下tokenizer返回的输入id，我们会发现它们与我们之前看到的有一点点不同:

In [4]:
sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)
print(model_inputs["input_ids"])

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102]
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]


一个令牌ID在开始时添加，另一个在结束时添加。让我们解码上面的两个id序列，看看这是怎么回事:

In [5]:
print(tokenizer.decode(model_inputs["input_ids"]))
print(tokenizer.decode(ids))

[CLS] i've been waiting for a huggingface course my whole life. [SEP]
i've been waiting for a huggingface course my whole life.


标记器在开头添加了特殊单词[CLS]，在结尾添加了特殊单词[SEP]。这是因为模型是用这些进行预训练的，所以为了得到相同的推断结果，我们也需要添加它们。注意，有些模型没有添加特殊的单词，或者添加不同的单词;模型也可以只在开头或结尾添加这些特殊的单词。在任何情况下，标记器都知道需要哪些标记，并将为您处理这些标记。

#### Wrapping up: From tokenizer to model

现在我们已经看到了tokenizer对象在应用于文本时使用的所有单独步骤，让我们最后一次看看它如何处理多个序列(填充!)，非常长的序列(截断!)，以及多种类型的张量与它的主要API:

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)

In [7]:
output

SequenceClassifierOutput(loss=None, logits=tensor([[-1.560701,  1.612286],
        [-3.618317,  3.913749]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)